In [1]:
%load_ext autoreload
%autoreload 2

import sys

linewidth = 100
formatter = {
    'float': lambda num: f'{num:5.1f}',
    'int': lambda num: f'{float(num):5.1f}',
}
print_args = {
#     'threshold': sys.maxsize,
    'threshold': 100,
    'linewidth': 200,
    'formatter': formatter,
    'edgeitems': 10,
}


In [2]:
import numpy as np
from qtmpy.constants import RYDBERG

alat = 5.107
latvec_alat = 0.5 * np.array([
    [ 1,  1,  1],
    [-1,  1,  1],
    [-1, -1,  1]
])

ecutwfn = 40 * RYDBERG
print(ecutwfn)

20.0


In [3]:
from qtmpy import qtmconfig
qtmconfig.fft_backend = 'pyfftw'

In [4]:
from qtmpy.lattice import RealLattice, ReciLattice

reallat = RealLattice.from_alat(alat, *latvec_alat)
recilat = ReciLattice.from_reallat(reallat)

print(reallat.axes_alat)
print(recilat.tpiba)
print(recilat.axes_tpiba)

([0.5, 0.5, 0.5], [-0.5, 0.5, 0.5], [-0.5, -0.5, 0.5])
1.2303084603837058
([1.0, 1.1102230246251565e-16, 1.0], [-1.0000000000000002, 1.0000000000000002, 8.695696296261264e-17], [0.0, -1.0, 1.0])


In [5]:
from qtmpy.gspace.gspc import GSpace

gspc = GSpace(recilat, 4 * ecutwfn)
print(gspc.grid_shape)
print(gspc.size_g)
with np.printoptions(**print_args):
    for i in range(3):
        print(f'G_{i+1}: ', gspc.g_cryst[i])
    print(f'G^2: ', gspc.g_norm2)

(18, 18, 18)
2243
G_1:  [  0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0 ...  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0]
G_2:  [  0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0 ...  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0]
G_3:  [  0.0   1.0   2.0   3.0   4.0   5.0   6.0   7.0  -7.0  -6.0 ...   5.0   6.0   7.0  -7.0  -6.0  -5.0  -4.0  -3.0  -2.0  -1.0]
G^2:  [  0.0   3.0  12.1  27.2  48.4  75.7 109.0 148.3 148.3 109.0 ...  78.7 112.0 151.4 151.4 112.0  78.7  51.5  30.3  15.1   6.1]


In [6]:
from qtmpy.containers import FieldG, FieldR

def print_info(f):
    typ = type(f)
    basis = getattr(f, 'basis_type', 'N\A')
    shape = getattr(f, 'shape', 'N\A')
    print(f"type: {str(type(f)):35}\nbasis: '{basis}', shape: {shape}\n")

# Creating a empty GField instance
print('empty')
a_g = FieldG.empty(gspc, (4, 10))
print_info(a_g)

# Converting to RField instance
print('rfield')
a_r = a_g.to_fieldr()
print_info(a_r)

# Indexing
print('indexing')
arr_idx = a_r[2]
print_info(arr_idx)

# Slicing
print('slicing')
arr_sl = a_r[:, :5]
print_info(arr_sl)

# Unpacking
print('unpacking')
x, y = a_r[0, :2]
print_info(x)
print_info(y)

# Binary operations
print('add')
z = x + y
print_info(z)

# Binary operations with broadcasting
print('add bcast')
z = x + a_r[:2,:3]
print_info(z)

# Operations between mismatched basis; raises exception
print('mismatch')
try:
    z = x + x.to_fieldg()
except TypeError as e:
    print(e)
## TypeError: mismatch in 'basis_type' between two Field instances

empty
type: <class 'qtmpy.containers.field.FieldG'>
basis: 'g', shape: (4, 10)

rfield
type: <class 'qtmpy.containers.field.FieldR'>
basis: 'r', shape: (4, 10)

indexing
type: <class 'qtmpy.containers.field.FieldR'>
basis: 'r', shape: (10,)

slicing
type: <class 'qtmpy.containers.field.FieldR'>
basis: 'r', shape: (4, 5)

unpacking
type: <class 'qtmpy.containers.field.FieldR'>
basis: 'r', shape: ()

type: <class 'qtmpy.containers.field.FieldR'>
basis: 'r', shape: ()

add
type: <class 'qtmpy.containers.field.FieldR'>
basis: 'r', shape: ()

add bcast
type: <class 'qtmpy.containers.field.FieldR'>
basis: 'r', shape: (2, 3)

mismatch
mismatch in 'basis_type' between two 'Buffer' instances.


In [7]:
a_g = FieldG.empty(gspc, (4, 10))
print_info(a_g)
a_g.g[:] = np.random.rand(*a_g.data.shape)

s = sum(a_g)
print_info(s)
print(np.allclose(s.data, np.sum(a_g.data, axis=0)))

s = sum(sum(a_g))
print_info(s)
print(np.allclose(s.data, np.sum(a_g.data, axis=(0, 1))))

type: <class 'qtmpy.containers.field.FieldG'>
basis: 'g', shape: (4, 10)

type: <class 'qtmpy.containers.field.FieldG'>
basis: 'g', shape: (10,)

True
type: <class 'qtmpy.containers.field.FieldG'>
basis: 'g', shape: ()

True


In [8]:
from qtmpy.pot.hartree import hartree

print(gspc.g_cart[0, gspc.idxsort])
dat = np.load('hart.npz')

# print('new')
# with np.printoptions(**print_args):
#     for i in range(3):
#         print(f'G_{i+1}: ', gspc.g_cryst[i][gspc.idxsort])
#     print(f'G^2: ', gspc.g_norm2[gspc.idxsort])
# print('-'*10)


g_cryst = dat['g_cryst']
g_cart = dat['g_cart']
g_norm2 = dat['g_norm2']
# print('reference')
# with np.printoptions(**print_args):
#     for i in range(3):
#         print(f'G_{i+1}: ', g_cryst[i])
#     print(f'G^2: ', g_norm2)    

print(np.allclose(g_cryst, gspc.g_cryst[:, gspc.idxsort]))
print(np.allclose(g_cart, gspc.g_cart[:, gspc.idxsort]))
print(np.allclose(g_norm2, gspc.g_norm2[gspc.idxsort]))

rho_g = FieldG.empty(gspc, 2)
rho_g.g[gspc.idxsort] = dat['rho_g']
print(np.allclose(dat['rho_g'], rho_g.data[:, gspc.idxsort]))

vhart_r = hartree(rho_g)
vhart_g = vhart_r.to_fieldg()
print(np.allclose(dat['v_g'], vhart_g.g[gspc.idxsort]))

[ 0.00000000e+00  2.22044605e-16 -1.23030846e+00 ...  7.38185076e+00
  2.46061692e+00  2.46061692e+00]
True
True
True
True
<class 'qtmpy.containers.field.FieldG'>
True


In [11]:
from qtmpy.gspace import GkSpace
from qtmpy.containers import GWavefun

nbnd = 10

gkspc = GkSpace(gspc, [0., 0., 0.])
wfn = GWavefun.empty(gkspc, nbnd)

ImportError: cannot import name 'GWavefun' from 'qtmpy.containers' (/home/lightpeak008/qtmpy/src/qtmpy/containers/__init__.py)

In [ ]:
import numpy as np
import pyfftw

a = np.empty(1000, dtype='c16')

print(pyfftw.simd_alignment)
print(pyfftw.is_byte_aligned(a))
print(np.dtype('c16').alignment)
print(np.dtype('f16').alignment)